## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/7/24

####  比赛官网如下:<a href="https://challenge.xfyun.cn/topic/info?type=short-video-recommendation&option=ssgy">短视频精准推荐挑战赛</a>,更多比赛的baseline可以在<a href="https://github.com/yunsuxiaozi/AI-and-competition">AI and competition</a>找到。

#### 注:这个比赛奖金不高,参加这个比赛是因为我从来没有参加过推荐的比赛,为了长长见识,本次比赛的评估指标我也是第一次看到。这个方案目前没有用任何模型,在排行榜上目前排4/10,故开源。

### 1.读取日志数据

In [1]:
import pandas as pd#导入csv文件的库
import numpy as np#矩阵运算与科学计算的库
path=''#这里需要换成你自己的数据路径
logs=pd.read_csv(path+"uid_click_log.csv")
print(f"len(logs):{len(logs)}")
#观影时长/视频时长,如果百分比小于1,说明没看完就走了,大于1说明有些地方重复观看了
logs['playpercent']=logs['playtime']/logs['duration']
logs.head()

len(logs):3487102


,uid,vid,cid,playtime,duration,date,rank,playpercent
0,100000,100870,100037,4966,9590,20200706,1,0.517831
1,100000,101167,100024,5037,4949,20200706,2,1.017781
2,100000,103608,100008,5137,14884,20200706,3,0.345136
3,100000,100220,100084,5139,16784,20200706,4,0.306184
4,100000,101674,100027,5149,5760,20200706,5,0.893924


### 如果一个视频少于5个人看,那就说明比较小众,也就不推荐了,这里就是把这些视频过滤掉。

In [2]:
vid_count=logs['vid'].value_counts().to_dict()

#如果一个视频少于5个人看,那就说明比较小众,也就不推荐了
dislike_vids=[]
for vid,count in vid_count.items():
    if count<5:
        dislike_vids.append(vid)
print(f"len(dislike_vids):{len(dislike_vids)}")    
logs=logs[~logs['vid'].isin(dislike_vids)]
print(f"len(logs):{len(logs)}")
logs.head()

len(dislike_vids):1328
len(logs):3484168


,uid,vid,cid,playtime,duration,date,rank,playpercent
0,100000,100870,100037,4966,9590,20200706,1,0.517831
1,100000,101167,100024,5037,4949,20200706,2,1.017781
2,100000,103608,100008,5137,14884,20200706,3,0.345136
3,100000,100220,100084,5139,16784,20200706,4,0.306184
4,100000,101674,100027,5149,5760,20200706,5,0.893924


### 这里看看每个视频大类cid里每个vid的观看人数count.

In [3]:
#每个cid里每个vid的count
cid_vid_count=logs.groupby(['cid','vid'])['uid'].count().reset_index().rename(columns={"uid":'count'})
#每个视频观看的人数越多排在越前面
cid_vid_count=cid_vid_count.sort_values(['cid','count'],ascending=[True,False])
cid_vid_count.head()

,cid,vid,count
9,100000,100098,2085
505,100000,103579,1909
452,100000,103217,1904
13,100000,100113,1895
310,100000,102112,1895


### 这里看看每个用户uid喜欢看每个cid的时间。

In [4]:
#每个uid最喜欢什么cid
uid_cid_playtime=logs.groupby(['uid','cid'])['playtime'].sum().reset_index().rename(columns={"playtime":'playtimesum'})
#看看每个uid最喜欢看哪个类型的cid
uid_cid_playtime=uid_cid_playtime.sort_values(['uid','playtimesum'],ascending=[True,False])
uid_cid_playtime.head()

,uid,cid,playtimesum
0,100000,100000,1193095
16,100000,100019,687648
22,100000,100027,672526
14,100000,100017,357325
3,100000,100005,319041


### 这里选择每个用户uid最喜欢看的3个cid中的2个(1个)vid。

In [5]:
submission=pd.read_csv(path+"example.csv")
print(f"len(submission):{len(submission)}")
uids=submission['uid'].unique()
test_videos=[]
for i in range(len(uids)):
    #初始化填充,纯粹就是选择看的人最多的5个视频作为推荐
    videos=[100592,102029,100423,100833,101428]
    uid=uids[i]
    #每个uid喜欢的cids
    cids=uid_cid_playtime[uid_cid_playtime['uid']==uid]['cid'].values
    fill_idx=0#从0开始填充
    #第二次填充,给每个uid推荐它们喜欢的cid下最受欢迎的视频,为了让视频丰富一点,每个cid下选2个视频,最后一个cid下选一个视频
    for cid in cids:
        #uid喜欢的cid里推荐2个uid
        vids=cid_vid_count[cid_vid_count['cid']==cid]['vid'].values
        if fill_idx<4:
            videos[fill_idx:fill_idx+2]=vids[:2]
        else:
            videos[fill_idx:fill_idx+1]=vids[:1]
        fill_idx+=2
        if fill_idx>5:
            break
    test_videos+=videos
submission['vid']=test_videos
submission.to_csv(path+"most_like.csv",index=None)
submission.head()

len(submission):25195


,uid,vid
0,100000,100098
1,100000,103579
2,100000,100019
3,100000,100515
4,100000,100981
